In [1]:
# 数据处理及导入导出
import pandas as pd  
# 爬虫
import requests  
import math  
import time
# 数据可视化
import seaborn as sns
import matplotlib.pyplot as plt
# 统计建模
import statsmodels.api as sm
# 词云
from wordcloud import WordCloud  
from imageio import imread
import jieba

In [ ]:
def get_json(url, num, position, city):  
    '''
        从网页获取 JSON，使用 POST 请求，加上头部信息
    '''
    # Request Headers
    my_headers = {  
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36',  
        'Host':'www.lagou.com',  
        'Referer':'https://www.lagou.com/jobs/list_%E6%95%B0%E6%8D%AE%E4%BA%A7%E5%93%81%E7%BB%8F%E7%90%86?city=' + city + '&cl=false&fromSearch=true&labelWords=&suginput=',  
        'X-Anit-Forge-Code':'0',  
        'X-Anit-Forge-Token': 'None',  
        'X-Requested-With':'XMLHttpRequest'  
    }

    # Form Data
    my_data = {  
        'first': 'true',  
        'pn': num,  
        'kd': position
    }  
    
    res = requests.post(url, headers = my_headers, data = my_data)  
    res.raise_for_status()  
    res.encoding = 'utf-8'  
    
    # 得到包含职位信息的字典  
    page = res.json()  
    return page  

In [2]:
def lagou_spider(position, city):  
    '''
        爬取拉勾职位列表数据
    '''
    # Request URL
    url = 'https://www.lagou.com/jobs/positionAjax.json?city=' + city + '&needAddtionalResult=false'  
    
    # 先设定页数为1,获取总的职位数  
    page_1 = get_json(url,1)  
    total_count = page_1['content']['positionResult']['totalCount']  
    num = get_page_num(total_count)  
    total_info = []  
    time.sleep(20)  
    print('职位总数:{},页数:{}'.format(total_count,num))  

    for n in range(1,num+1):  
        # 对每个网页读取JSON, 获取每页数据  
        page = get_json(url,n)  
        jobs_list = page['content']['positionResult']['result']  
        page_info = get_page_info(jobs_list)  
        total_info += page_info  
        print('已经抓取第{}页, 职位总数:{}'.format(n, len(total_info)))  
        # 每次抓取完成后,暂停一会,防止被服务器拉黑  
        time.sleep(30)  
       
    #将总数据转化为data frame再输出  
    df = pd.DataFrame(data = total_info,columns = ['公司全名','公司简称','公司规模','融资阶段','区域','职位名称','工作经验','学历要求','工资','职位福利'])   
    df.to_csv('spider_data.csv',index = False)  
    
    print('已保存为csv文件.')  